In [1]:
##########################################
#  STEP 1. IMPORTS & GLOBAL CONFIG       #
##########################################
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import parselmouth

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Directory where your data is located (adjust as needed)
BASE_DIR = r"/Users/shreyanandini/Desktop/Stress Level Detection From Speech/Audio_Speech_Actors_01-24"

/Users/shreyanandini/Desktop/Stress Level Detection From Speech/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
BASE_DIR = r"/Users/shreyanandini/Desktop/Stress Level Detection From Speech/Audio_Speech_Actors_01-24"
total_files = sum(len(files) for _, _, files in os.walk(BASE_DIR))

print(f"Total number of files: {total_files}")

Total number of files: 1421


In [3]:
# ##########################################
# #  STEP 2. DATA LOADING & LABELING       #
# ##########################################
# # We create a DataFrame to store metadata
# data_df = pd.DataFrame(columns=['file_path', 'src', 'actor', 'gender', 'intensity', 'statement', 'repeat', 'emotion','transcript'])

# # Check if the base directory exists
# if not os.path.isdir(BASE_DIR):
#     raise ValueError(f"BASE_DIR does not exist: {BASE_DIR}")

# dir_list = os.listdir(BASE_DIR)
# dir_list.sort()
# count = 0
# recognizer = sr.Recognizer()

# def extract_pitch(file_path):
#     try:
#         y, sr = librosa.load(file_path, sr=None)
#         pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
#         pitches = pitches[np.nonzero(pitches)]  # Filter non-zero pitch values
#         if len(pitches) > 0:
#             return np.mean(pitches)
#         else:
#             return 0
#     except Exception as e:
#         print(f"Error extracting pitch for {file_path}: {e}")
#         return None
    
# base_directory = "/Users/shreyanandini/Desktop/Stress Level Detection From Speech/Audio_Speech_Actors_01-24"
# output_data = []

# # Traverse through the directories
# for subdir in sorted(os.listdir(base_directory)):
#     subdir_path = os.path.join(base_directory, subdir)
#     if os.path.isdir(subdir_path):
#         for file in os.listdir(subdir_path):
#             if file.endswith(".wav"):
#                 file_path = os.path.join(subdir_path, file)
#                 try:
#                     # Extract features
#                     pitch = extract_pitch(file_path)
                    
#                     # Append results to the data list
#                     output_data.append({
#                         "file_path": subdir_path+'/'+file,
#                         "Pitch": pitch if pitch is not None else 0
#                     })
#                     #print(f"Processed {file_path}")
#                 except Exception as e:
#                     print(f"Error processing {file_path}: {e}")

# Convert the data to a DataFrame and save as CSV
# output_df = pd.DataFrame(output_data)
# output_df.to_csv("audio_to_pitch.csv", index=False)
# print("Feature extraction complete. Results saved to audio_to_pitch.csv.")

# for folder in dir_list:
#     folder_path = os.path.join(BASE_DIR, folder)
#     transcript = ""
#     # If it's not a directory, skip
#     if not os.path.isdir(folder_path):
#         continue
#     for f in os.listdir(folder_path):
#         if not f.endswith('.wav'):
#             continue      
#         # Example filename pattern: "03-01-01-01-01-01-01.wav"
#         nm = f.split('.')[0].split('-')         
#         #try:
#         emotion = int(nm[2])   
#         src = int(nm[1])
#         actor = int(nm[-1])
#         emotion = int(nm[2])
        
#         if int(actor)%2 == 0:
#             gender = "female"
#         else:
#             gender = "male"
        
#         if nm[3] == '01':
#             intensity = 0
#         else:
#             intensity = 1
        
#         if nm[4] == '01':
#             statement = 0
#         else:
#             statement = 1
        
#         if nm[5] == '01':
#             repeat = 0
#         else:
#             repeat = 1
#         file_path = os.path.join(folder_path, f)
#         try:
#             with sr.AudioFile(file_path) as source:
#                 data = recognizer.record(source)
#                 transcript = recognizer.recognize_google(data, key=None)
#             #print("Transcript:", transcript)
#         except Exception as e:
#             print("Error during transcription:", e)
#             transcript = "Could not process audio transcription."
#             # except (IndexError, ValueError) as e:
#             #     print(f"Skipping file (naming mismatch): {f}")
#             #     continue
#             #print(file_path)
#             #Ensuring the .wav file exists
#         if os.path.isfile(file_path):
#             data_df.loc[count] = [file_path, src, actor, gender, intensity, statement, repeat, emotion,transcript]
#             count += 1
#         else:
#             print(f"File not found: {file_path}")

In [3]:
df1 = pd.read_csv("audio_to_transcript.csv")
df2 = pd.read_csv('audio_to_pitch.csv')

df1_sorted = df1.sort_values(by='file_path').reset_index(drop=True)
df2_sorted = df2.sort_values(by='file_path').reset_index(drop=True)

# Merge the DataFrames on the 'file_path' column
data_df = pd.merge(df1_sorted, df2_sorted, on='file_path', how='inner')

# Save the result to a new CSV file if needed
data_df.to_csv('audio_voice_features.csv', index=False)

# Show the merged DataFrame
data_df


,Unnamed: 0.1,Unnamed: 0,file_path,src,actor,gender,intensity,statement,repeat,emotion,transcript,stress_label,Pitch
0,1094,19,/Users/shreyanandini/Desktop/Stress Level Dete...,1,1,male,0,0,0,1,can you talking by the door,0,1779.4193
1,608,55,/Users/shreyanandini/Desktop/Stress Level Dete...,1,1,male,0,0,1,1,can you talking by the door,0,1818.3252
2,277,31,/Users/shreyanandini/Desktop/Stress Level Dete...,1,1,male,0,1,0,1,dogs are sitting by the door,0,1828.8231
3,1368,13,/Users/shreyanandini/Desktop/Stress Level Dete...,1,1,male,0,1,1,1,talk to city by the door,0,1784.7290
4,254,56,/Users/shreyanandini/Desktop/Stress Level Dete...,1,1,male,0,0,0,2,kids talking by the door,0,1804.7471
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1361,1398,/Users/shreyanandini/Desktop/Stress Level Dete...,1,24,female,0,1,1,8,don't just sitting by the door,0,1940.3499
1416,825,1373,/Users/shreyanandini/Desktop/Stress Level Dete...,1,24,female,1,0,0,8,kids are talking by the door,0,1908.4088
1417,812,1399,/Users/shreyanandini/Desktop/Stress Level Dete...,1,24,female,1,0,1,8,kids are talking about the door,0,2001.0757
1418,972,1419,/Users/shreyanandini/Desktop/Stress Level Dete...,1,24,female,1,1,0,8,talk setting by the door,0,1897.3414


In [4]:
def map_emotion_to_stress(e):
    """
    In your dataset, 4/6 = high stress -> 2, 
                     5/7 = low stress -> 1,
                    else 0 = no stress
    """
    if e in [4,6]:
        return 2
    elif e in [5,7]:
        return 1
    else:
        return 0

data_df['stress_label'] = data_df['emotion'].apply(map_emotion_to_stress)

print(data_df.head())
print("Total files:", len(data_df))

# Shuffle dataframe
data_df = data_df.sample(frac=1.0, random_state=RANDOM_SEED).reset_index(drop=True)

   Unnamed: 0.1  Unnamed: 0  \
0          1094          19   
1           608          55   
2           277          31   
3          1368          13   
4           254          56   

                                           file_path  src  actor gender  \
0  /Users/shreyanandini/Desktop/Stress Level Dete...    1      1   male   
1  /Users/shreyanandini/Desktop/Stress Level Dete...    1      1   male   
2  /Users/shreyanandini/Desktop/Stress Level Dete...    1      1   male   
3  /Users/shreyanandini/Desktop/Stress Level Dete...    1      1   male   
4  /Users/shreyanandini/Desktop/Stress Level Dete...    1      1   male   

   intensity  statement  repeat  emotion                    transcript  \
0          0          0       0        1   can you talking by the door   
1          0          0       1        1   can you talking by the door   
2          0          1       0        1  dogs are sitting by the door   
3          0          1       1        1      talk to city by the 

In [5]:
##########################################
#  STEP 3. TRAIN/VAL/TEST SPLIT          #
##########################################
train_df, temp_df = train_test_split(
    data_df,
    test_size=0.2,
    stratify=data_df['stress_label'],
    random_state=RANDOM_SEED
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['stress_label'],
    random_state=RANDOM_SEED
)

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")

Train size: 1136, Val size: 142, Test size: 142


In [6]:
##########################################
#  STEP 4. FEATURE EXTRACTION (MFCC)     #
##########################################
SAMPLE_RATE = 16000       # or 22050
MAX_DURATION_SEC = 5.0    # we’ll pad/truncate to 5s
N_MFCC = 40
FIXED_FRAMES = 200        # number of frames along time axis

def load_audio(file_path, sr=SAMPLE_RATE, max_duration=MAX_DURATION_SEC):
    # Load up to max_duration seconds
    y, sr = librosa.load(file_path, sr=sr, duration=max_duration)
    return y, sr

def extract_mfcc(file_path, sr=SAMPLE_RATE, n_mfcc=N_MFCC):
    y, sr = load_audio(file_path, sr=sr, max_duration=MAX_DURATION_SEC)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # shape: (n_mfcc, time_frames)

    # Pad or truncate to FIXED_FRAMES along the time axis
    if mfcc.shape[1] < FIXED_FRAMES:
        pad_width = FIXED_FRAMES - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0,0), (0,pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :FIXED_FRAMES]
    return mfcc

def build_features_and_labels(df):
    X_list, y_list = [], []

    for _, row in df.iterrows():
        file_path = row['file_path']
        label = row['stress_label']

        if not os.path.isfile(file_path):
            print(f"Warning: file not found -> {file_path}. Skipping.")
            continue

        # Extract MFCC
        mfcc = extract_mfcc(file_path)
        # Expand dims for CNN: (n_mfcc, time_frames) -> (n_mfcc, time_frames, 1)
        mfcc = np.expand_dims(mfcc, axis=-1)

        # Process Pitch
        pitch = np.array([[row['Pitch']]])  # Shape: (1, 1)
        pitch_repeated = np.repeat(pitch, repeats=mfcc.shape[1], axis=1)  # Shape: (1, FIXED_FRAMES)
        pitch_repeated = np.expand_dims(pitch_repeated, axis=-1)  # Shape: (1, FIXED_FRAMES, 1)

        # Combine MFCC and Pitch along the feature axis
        combined_features = np.concatenate((mfcc, pitch_repeated), axis=0)  # Shape: (n_mfcc + 1, FIXED_FRAMES, 1)

        X_list.append(combined_features)
        y_list.append(label)

    X_arr = np.array(X_list, dtype=np.float32)
    y_arr = np.array(y_list, dtype=np.int64)

    return X_arr, y_arr



# Build train/val/test feature sets
print("Extracting features for TRAIN set...")
X_train, y_train = build_features_and_labels(train_df)

print("Extracting features for VAL set...")
X_val, y_val = build_features_and_labels(val_df)

print("Extracting features for TEST set...")
X_test, y_test = build_features_and_labels(test_df)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


Extracting features for TRAIN set...
Extracting features for VAL set...
Extracting features for TEST set...
X_train shape: (1136, 41, 200, 1)
y_train shape: (1136,)


In [7]:
num_classes = 3
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_val_one_hot = to_categorical(y_val, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

In [8]:
##########################################
#  STEP 5. BUILD THE CNN MODEL           #
##########################################
num_classes = 3  

model = models.Sequential([
    layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(N_MFCC, FIXED_FRAMES, 1)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(12288, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0000001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/Users/shreyanandini/Desktop/Stress Level Detection From Speech/venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 198, 16)    │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 99, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 97, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 48, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 12288)          │   151,007,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │        36,867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,048,899 (576.21 MB)

 Trainable params: 151,048,899 (576.21 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
##########################################
#  STEP 6. TRAIN THE MODEL               #
##########################################
EPOCHS = 50     # adjust as needed
BATCH_SIZE = 64
early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor (e.g., validation loss)
    patience=5,              # Number of epochs to wait before stopping
    restore_best_weights=True # Restore the best weights
)

history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping]  # Add the early stopping callback here
)



Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 754ms/step - accuracy: 0.4038 - loss: 5.2942 - val_accuracy: 0.4789 - val_loss: 2.5798
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 772ms/step - accuracy: 0.3795 - loss: 5.4598 - val_accuracy: 0.4859 - val_loss: 2.3365
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 733ms/step - accuracy: 0.4017 - loss: 5.3031 - val_accuracy: 0.4930 - val_loss: 2.2351
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 707ms/step - accuracy: 0.4235 - loss: 5.0342 - val_accuracy: 0.5070 - val_loss: 2.2309
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 149s 9s/step - accuracy: 0.4424 - loss: 4.8414 - val_accuracy: 0.4859 - val_loss: 2.2819
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 448ms/step - accuracy: 0.4385 - loss: 4.9193 - val_accuracy: 0.5141 - val_loss: 2.2083
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 431ms/step - accuracy: 0.3992 - loss: 5.2107 - val_accuracy: 0.5141 - val_loss: 2.1963
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 422ms/step - accuracy: 0.3880 - loss: 5.2774 - val_accuracy:

In [10]:
##########################################
#  STEP 7. EVALUATION & METRICS          #
##########################################
test_loss, test_acc = model.evaluate(X_test, y_test_one_hot, verbose=0)
print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Stress', 'Low Stress','High Stress']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Test Loss: 2.7387 | Test Accuracy: 0.4648
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step

Classification Report:
              precision    recall  f1-score   support

   No Stress       0.51      0.72      0.60        67
  Low Stress       0.48      0.43      0.46        37
 High Stress       0.13      0.05      0.08        38

    accuracy                           0.46       142
   macro avg       0.38      0.40      0.38       142
weighted avg       0.40      0.46      0.42       142

Confusion Matrix:
[[48 10  9]
 [17 16  4]
 [29  7  2]]


In [11]:
# ---------------------------
# 7. Evaluate
# ---------------------------
loss, acc = model.evaluate(X_test, y_test_one_hot, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {acc*100:.2f}%")


Test Loss: 2.7387, Test Accuracy: 46.48%


In [16]:
##########################################
#  STEP 8. SAVE THE MODEL                #
##########################################
model.save("stress_detector_cnn.h5")
print("Model saved as stress_detector_cnn.h5")

# Optionally, you can also do:
# joblib.dump(model, "stress_detector_cnn.pkl")
# if you prefer a pickle format.

Model saved as stress_detector_cnn.h5


In [13]:
import pickle
pickle.dump(model,open('model.pkl','wb'))